# Loads dependencies

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, spline_filter

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import time
import wandb

In [30]:
#BASELINE
#BreakoutDeterministic-v4-Basline -> uctalzu1 *
#SpaceInvaders-v0-Baseline -> uzn61jbj *
#Assault-v0-Baseline -> 6v3bnwyv *
#MsPacman-v0-Baseline -> v3mzfrpy *
#BeamRider-v0-Baseline -> jtwu6mg5 *
#Seaquest-v0-Baseline -> ou3wa1y2 *
#Alien-v0-Baseline -> xrn3calz *

#MODEL INJECTION
#BreakoutDeterministic-v4-Inject-BreakoutDeterministiv-v4 -> ymsiskbx *
#MsPacman-v0-Inject-Assault-v0 -> 18e1bb7c *
#Seaquest-v0-Inject-Assault-v0 -> hdg1pn88 *
#Alien-v0-Inject-Seaquest-v0 -> ipzn90am *

#Q INJECTION
#BreakoutDeterministic-v4-QInject -> q1ny8ouw*
#MsPacman-v0-QInject -> 69slxq33*
#Seaquest-v0-QInject -> h4ie3bvf*
#Alien-v0-QInject -> sl8yme9p*

#UNCOMMENT FOR CSV DOWNLOAD

#api = wandb.Api()
#run = api.run("mheinz02/DQN/sl8yme9p")
#metrics_dataframe = run.history()
#savename = './csv_files/'+'Alien-v0-QInject.csv'
#metrics_dataframe.to_csv(savename)

# Thesis
- reward_mean x:episode
- episode_reward x:episode
- sum_reward x:episode

# Appendix
- mae x:episode
- loss x:episode
- duration x:episode
- episode_steps x:episode
- step x:episode
- mean_q x:episode
- mean_eps x:episode

In [31]:
#BASELINE
csv_breakout_baseline = './csv_files/BreakoutDeterministic-v4-Baseline.csv'
csv_space_invaders_baseline = './csv_files/SpaceInvaders-v0-Baseline.csv'
csv_assault_baseline = './csv_files/Assault-v0-Baseline.csv'
csv_pacman_baseline = './csv_files/MsPacman-v0-Baseline.csv'
csv_seaquest_baseline = './csv_files/Seaquest-v0-Baseline.csv'
csv_alien_baseline = './csv_files/Alien-v0-Baseline.csv'

#MODEL INJECTION
csv_pacman_assault = './csv_files/MsPacman-v0-Inject-Assault.csv'
csv_breakout_breakout = './csv_files/BreakoutDeterministic-v4-Inject-BreakoutDeterministiv-v4.csv'
csv_seaquest_assault = './csv_files/Seaquest-v0-Inject-Assault-v0.csv'
csv_alien_seaquest = './csv_files/Alien-v0-Inject-Seaquest-v0.csv'

#Q INJECTION
csv_q_pacman = './csv_files/MsPacman-v0-QInject.csv'
csv_q_breakout = './csv_files/BreakoutDeterministic-v4-QInject.csv'
csv_q_seaquest = './csv_files/Seaquest-v0-QInject.csv'
csv_q_alien = './csv_files/Alien-v0-QInject.csv'

In [60]:
#GAME Baseline OR GAme Inject(Other Game)
#breakout
h = AnalyzeNew(csv1=csv_seaquest_baseline,
            csv2=csv_q_seaquest,
            game1='Seaquest_Baseline',
            game2='Seaquest_QInject')

In [62]:
#h.getStatsThesis()

In [55]:
#h.getChartsThesis(21,1)
#h.getChartsAppendix()

In [5]:
class AnalyzeNew():
    """
    This agent analyzes the training results. Imports the csv and 
    compares the data.
    """
    def __init__(self, csv1=None, csv2=None,game1=None,game2=None):
        self.csv1DF = None
        self.csv2DF = None
        
        try:
            self.csv1DF = pd.read_csv(csv1)
        except:
            pass
        try:
            self.csv2DF = pd.read_csv(csv2)
        except:
            pass

        self.game1 = game1
        self.game2 = game2
        
        self.xAxis = 'episode'
        
        self.logsThesis = {'Reward Mean':'reward_mean',
                           'Episode Reward':'episode_reward',
                           'Sum Reward':'sum_reward',
                           'Mean Q':'mean_q',
                           'Mean Epsilon':'mean_eps'}
        
        self.logsAppendix = {'Loss':'loss',
                             'Duration':'duration',
                             'Episode Steps':'episode_steps',
                             'Step':'step'}
        
    def getStatsThesis(self):
        """
        Calculates the stats that are required.
        """
        if self.csv1DF is not None:
            print(f"Game: {self.game1}")
            for log in self.logsThesis:
                print(f"Stats for: {log}")
                print(self.csv1DF.loc[:,self.logsThesis[log]].describe())
                print("------")
            
        print()
        if self.csv2DF is not None:
            print(f"Game: {self.game2}")
            for log in self.logsThesis:
                print(f"Stats for: {log}")
                print(self.csv2DF.loc[:,self.logsThesis[log]].describe())
                print("------")
    
    def getChartsThesis(self,window_length,polyorder):
        """
        Creates the charts for the thesis.
        """
        xAxis = 'step'
        xAxisLabel = 'Step'
        color_gray= '#979797'
        color_red = '#FF0000'
        #xAxisStep = 'episode'
        if self.csv2DF is None:
            fig = plt.figure(figsize=(15,15))
            title = f"{self.game1}"
            fig.suptitle(title,fontsize = 16)
            grid = fig.add_gridspec(2,2)
            
            #sum_reward
            axSumReward = fig.add_subplot(grid[0,:])
            x = self.csv1DF.loc[:,xAxis]
            y = self.csv1DF.loc[:,'sum_reward']
            axSumReward.plot(x,y,color=color_gray)
            axSumReward.set(title='Sum Reward',
                            ylabel="Sum of Reward",
                            xlabel=xAxisLabel)

            #episode_reward
            axEpisodeReward = fig.add_subplot(grid[1,0])
            x = self.csv1DF.loc[:,xAxis]
            y = self.csv1DF.loc[:,'episode_reward']
            
            axEpisodeReward.plot(x,y,color=color_gray,label=self.game1,alpha=0.2)
            axEpisodeReward.plot(x,savgol_filter(y,window_length,polyorder),color=color_gray,label=self.game1+' smoothed')
            
            
            axEpisodeReward.set(title='Episode Reward',
                            ylabel="Episode Reward",
                            xlabel=xAxisLabel)
            
            #reward_mean
            axRewardMean = fig.add_subplot(grid[1,1])
            x = self.csv1DF.loc[:,xAxis]
            y = self.csv1DF.loc[:,'reward_mean']
            
            axRewardMean.plot(x,y,color=color_gray,label=self.game1,alpha=0.2)
            axRewardMean.plot(x,savgol_filter(y,window_length,polyorder),color=color_gray,label=self.game1+' smoothed')
            
            axRewardMean.set(title='Reward Mean',
                            ylabel="Reward Mean",
                            xlabel=xAxisLabel)
            
            
            grid.tight_layout(fig,rect=[0, 0.03, 1, 0.95])
            time_save = str(int(time.time()))
            savename = './Charts/'+self.game1+'.png'
            plt.savefig(savename)
        else:
            fig = plt.figure(figsize=(15,15))
            title = f"{self.game1} vs. {self.game2}"
            fig.suptitle(title, fontsize = 16)
            grid = fig.add_gridspec(2,2)
            
            #sum_reward
            axSumReward = fig.add_subplot(grid[0,:])
            x1 = self.csv1DF.loc[:,xAxis]
            x2 = self.csv2DF.loc[:,xAxis]
            y1 = self.csv1DF.loc[:,'sum_reward']
            y2 = self.csv2DF.loc[:,'sum_reward']
            
            #game1
            axSumReward.plot(x1,y1,label=self.game1,color=color_gray)
            
            #game2
            axSumReward.plot(x2,y2,label=self.game2,color=color_red)
            
            axSumReward.set(title='Sum Reward',
                            ylabel="Sum of Reward",
                            xlabel=xAxisLabel)
            axSumReward.legend(loc='best')

            
            
            #episode_reward
            axEpisodeReward = fig.add_subplot(grid[1,0])
            x1 = self.csv1DF.loc[:,xAxis]
            x2 = self.csv2DF.loc[:,xAxis]
            
            y1 = self.csv1DF.loc[:,'episode_reward']
            y2 = self.csv2DF.loc[:,'episode_reward']
            
            #game1
            axEpisodeReward.plot(x1,y1,label=self.game1,color=color_gray,alpha=0.2)
            axEpisodeReward.plot(x1,savgol_filter(y1,window_length,polyorder),label=self.game1+' smoothed',color=color_gray)
            
            #game2
            axEpisodeReward.plot(x2,y2,label=self.game2,color=color_red,alpha=0.2)
            axEpisodeReward.plot(x2,savgol_filter(y2,window_length,polyorder),label=self.game2+' smoothed',color=color_red)

            
            axEpisodeReward.set(title='Episode Reward',
                            ylabel="Episode Reward",
                            xlabel=xAxisLabel)
            axEpisodeReward.legend(loc='best')
            
            
            
            #reward_mean
            axRewardMean = fig.add_subplot(grid[1,1])
            x1 = self.csv1DF.loc[:,xAxis]
            x2 = self.csv2DF.loc[:,xAxis]
            
            y1 = self.csv1DF.loc[:,'reward_mean']
            y2 = self.csv2DF.loc[:,'reward_mean']
            
            #game1
            axRewardMean.plot(x1,y1,label=self.game1,color=color_gray,alpha=0.2)
            axRewardMean.plot(x1,savgol_filter(y1,window_length,polyorder),label=self.game1+' smoothed',color=color_gray)


            
            #game2
            axRewardMean.plot(x2,y2,label=self.game2,color=color_red,alpha=0.2)
            axRewardMean.plot(x2,savgol_filter(y2,window_length,polyorder),label=self.game2+' smoothed',color=color_red)


            
            axRewardMean.set(title='Reward Mean',
                            ylabel="Reward Mean",
                            xlabel=xAxisLabel)
            axRewardMean.legend(loc='best')
            
            
            grid.tight_layout(fig,rect=[0, 0.03, 1, 0.95])
            time_save = str(int(time.time()))
            savename = ('./Charts/'+self.game1+'_'+
                        self.game2+'.png')
            plt.savefig(savename)
            
    def getChartsAppendix(self):
        xAxis = 'step'
        xAxisLabel = 'Step'
        color_gray= '#979797'
        color_red = '#FF0000'
        #xAxisStep = 'episode'
        if self.csv2DF is None:
            fig = plt.figure(figsize=(15,20))
            title = f"Additional Metrics: {self.game1}"
            fig.suptitle(title,fontsize = 16)
            grid = fig.add_gridspec(6,1)
            
            #loss
            axLoss = fig.add_subplot(grid[0,0])
            x = self.csv1DF.loc[:,'step']
            y = self.csv1DF.loc[:,'loss']
            axLoss.plot(x,y,color=color_gray)
            axLoss.set(title='Loss',
                            ylabel="Loss",
                            xlabel='Steps')

            #duration
            axEpisodeDuration = fig.add_subplot(grid[1,0])
            x = self.csv1DF.loc[:,'episode']
            y = self.csv1DF.loc[:,'duration']
            
            axEpisodeDuration.plot(x,y,color=color_gray,label=self.game1)            
            
            axEpisodeDuration.set(title='Episode Duration',
                            ylabel="Duration (s)",
                            xlabel='Episode')
            
            #episode steps
            axEpisodeSteps = fig.add_subplot(grid[2,0])
            x = self.csv1DF.loc[:,'episode']
            y = self.csv1DF.loc[:,'episode_steps']
            
            axEpisodeSteps.plot(x,y,color=color_gray,label=self.game1)
            
            axEpisodeSteps.set(title='Episode Steps',
                            ylabel="Steps",
                            xlabel='Episode')
            
            #step
            axStep = fig.add_subplot(grid[3,0])
            x = self.csv1DF.loc[:,'episode']
            y = self.csv1DF.loc[:,'step']
            axStep.plot(x,y,color=color_gray)
            axStep.set(title='Steps',
                            ylabel="Steps",
                            xlabel='Episode')
            
            #mean_q
            axMeanQ = fig.add_subplot(grid[4,0])
            x = self.csv1DF.loc[:,xAxis]
            y = self.csv1DF.loc[:,'mean_q']
            axMeanQ.plot(x,y,color=color_gray)
            axMeanQ.set(title='Mean Q',
                            ylabel="Mean Q",
                            xlabel=xAxisLabel)
            
            #mean_eps
            axMeanEps = fig.add_subplot(grid[5,0])
            x = self.csv1DF.loc[:,xAxis]
            y = self.csv1DF.loc[:,'mean_eps']
            axMeanEps.plot(x,y,color=color_gray)
            axMeanEps.set(title='Mean Epsilon',
                            ylabel="Mean Epsilon",
                            xlabel=xAxisLabel)
            

            grid.tight_layout(fig,rect=[0, 0.03, 1, 0.95])
            time_save = str(int(time.time()))
            savename = './Charts/'+self.game1+'_app.png'
            plt.savefig(savename)
        else:
            fig = plt.figure(figsize=(15,20))
            title = f"Additional Metrics: {self.game1} vs. {self.game2}"
            fig.suptitle(title, fontsize = 16)
            grid = fig.add_gridspec(6,1)
            
            #loss
            axLoss = fig.add_subplot(grid[0,0])
            x1 = self.csv1DF.loc[:,'step']
            y1 = self.csv1DF.loc[:,'loss']
            
            x2 = self.csv2DF.loc[:,'step']
            y2 = self.csv2DF.loc[:,'loss']

            axLoss.plot(x1,y1,color=color_gray,label=self.game1)
            axLoss.plot(x2,y2,color=color_red,label=self.game2)
            axLoss.set(title='Loss',
                            ylabel="Loss",
                            xlabel='Steps')
            axLoss.legend(loc='best')
            
            #step
            axSteps = fig.add_subplot(grid[1,0])
            x1 = self.csv1DF.loc[:,'episode']
            y1 = self.csv1DF.loc[:,'step']
            
            x2 = self.csv2DF.loc[:,'episode']
            y2 = self.csv2DF.loc[:,'step']

            axSteps.plot(x1,y1,color=color_gray,label=self.game1)
            axSteps.plot(x2,y2,color=color_red,label=self.game2)
            axSteps.set(title='Steps',
                            ylabel="Steps",
                            xlabel='Episode')
            
            axSteps.legend(loc='best')
            
            
            #duration
            axDuration = fig.add_subplot(grid[2,0])
            x1 = self.csv1DF.loc[:,'episode']
            y1 = self.csv1DF.loc[:,'duration']
            
            x2 = self.csv2DF.loc[:,'episode']
            y2 = self.csv2DF.loc[:,'duration']

            axDuration.plot(x1,y1,color=color_gray,label=self.game1)
            axDuration.plot(x2,y2,color=color_red,label=self.game2)
            axDuration.set(title='Episode Duration',
                            ylabel="Duration (s)",
                            xlabel='Episode')
            axDuration.legend(loc='best')
            
            #episode steps
            axEpisodeSteps = fig.add_subplot(grid[3,0])
            x1 = self.csv1DF.loc[:,'episode']
            y1 = self.csv1DF.loc[:,'episode_steps']
            
            x2 = self.csv2DF.loc[:,'episode']
            y2 = self.csv2DF.loc[:,'episode_steps']

            axEpisodeSteps.plot(x1,y1,color=color_gray,label=self.game1)
            axEpisodeSteps.plot(x2,y2,color=color_red,label=self.game2)
            axEpisodeSteps.set(title='Episode Steps',
                            ylabel="Steps",
                            xlabel='Episode')
            axEpisodeSteps.legend(loc='best')
            
            
            #mean_q
            axMeanQ = fig.add_subplot(grid[4,0])
            x1 = self.csv1DF.loc[:,xAxis]
            x2 = self.csv2DF.loc[:,xAxis]
            y1 = self.csv1DF.loc[:,'mean_q']
            y2 = self.csv2DF.loc[:,'mean_q']
            axMeanQ.plot(x1,y1,label=self.game1,color=color_gray)
            axMeanQ.plot(x2,y2,label=self.game2,color=color_red)
            axMeanQ.set(title='Mean Q',
                            ylabel="Mean Q",
                            xlabel=xAxisLabel)
            axMeanQ.legend(loc='best')
            
            #mean_eps
            axMeanEps = fig.add_subplot(grid[5,0])
            x1 = self.csv1DF.loc[:,xAxis]
            x2 = self.csv2DF.loc[:,xAxis]
            y1 = self.csv1DF.loc[:,'mean_eps']
            y2 = self.csv2DF.loc[:,'mean_eps']
            axMeanEps.plot(x1,y1,label=self.game1,color=color_gray)
            axMeanEps.plot(x2,y2,label=self.game2,color=color_red)
            axMeanEps.set(title='Mean Epsilon',
                            ylabel="Mean Epsilon",
                            xlabel=xAxisLabel)
            axMeanEps.legend(loc='best')
            
            
            grid.tight_layout(fig,rect=[0, 0.03, 1, 0.95])
            time_save = str(int(time.time()))
            savename = ('./Charts/'+self.game1+'_'+
                        self.game2+'_app.png')
            plt.savefig(savename)